# Feature Engineering

### Main flavors of data and feature engineering
* Tabular: Dataframe model
    * "Typical" business data tables
* Batch/Tensor/Vector: Array model
    * Numeric data, timeseries, scientific data, audio, images, video, geodata, etc.
* Natural language
    * Batches of strings
    * Transformed into array data through NLP-specific techniques
    
<img src='images/flow-transform.png' width=800>

### "Must-haves" for feature engineering on large data

* Some data representation for the large dataset
    * Likely distributed, out-of-core, lazy, streaming, etc.
* Mechanism to load data from standard formats and locations into the representation
    * E.g., loading HDF5 in S3 or Parquet in HDFS
* APIs to apply feature engineering transformations
    * Mathematical operations
    * String, date, etc.
    * Custom ("user-defined")
* Integration to a modeling framework and/or ability to write to standard formats

### "Nice-to-haves"

* Intuitive data representation: similar to "small data" tooling
* APIs that resemble those of the most common industry-standard libraries
* Both modeling integration *and* ability to write out transformed data

FIXME: image

## Rise of Python

Python has become the *lingua franca* or dominant cross-cutting language for data science.

>
> __Note__ this is not to imply Python is the best or only language, or that other languages might not be intrinsically better or even, in the future, more successful. 
>
> There are wonderful things to be said for languages from Rust to R to Julia to many others, but for baseline data science capability and versatility in commercial enterprises today, it's Python
>

So we can turn to Python and look at the dominant libraries and tools within that ecosystem
* Tabular data: Pandas
* Array data: NumPy and derivatives like CuPy, JAX.numpy, etc.
* Basic modeling: scikit-learn, XGBoost, etc.
* Deep learning: PyTorch, Tensorflow
* NLP: SpaCy, NLTK, Huggingface, etc.

As we get into further parts of the workflow, like hyperparameter tuning or reinforcement learning there are more choices. 

For time reasons, we're going to stick to this core workflow of extraction through modeling and tuning, and not continue on into MLOps and deploment architectures, or meta-modeling platforms for experimentation, feature and provenance tracking, etc. That would be a bit too much to take on!

__Bottom line__: We want a data representation and APIs that are fairly close to the Pandas / NumPy / scikit-learn (SciPy) workflow. And we want elegant bridges into things like PyTorch, XGBoost, NLP tools, and tuning tools.

## Dask: SciPy at Scale

Luckily, Dask is well placed to solve this problem. 

While enterprises were still wrestling with JVM-based tools over the past 5 years, scientists, researchers, and others in the PyData and SciPy communities were building Dask, a pure-Python distributed compute platform that integrates deeply with all of the standard SciPy tools.

__What does this mean?__

We can take many of our local workflows to large-scale data via Dask with fairly minimal effort -- because under the hood, Dask is designed to use those "small data" structures in federation to create arbitrarily large ones.

FIXME: Dask DF and Array images




As an added bonus, due to the Dask architecture, it can leverage GPU-enabled versions of the underlying libraries.
* GPU + NumPy => CuPy
* GPU + Pandas => cuDF (RAPIDS CUDA dataframe)
* GPU + scikit-learn => cuML (RAPIDS CUDA algorithms)
etc.

### Using Dask for Feature Transformation

* We need to be able to load data in a standard format
* Manipulate it using dataframe or array APIs
* Write it and/or pass it efficiently to a modeling framework

In [2]:
from dask import dataframe as ddf
from dask import array as da
from dask.distributed import Client

client = Client(n_workers=2, threads_per_worker=1, memory_limit='1GB')

In [3]:
client

Client Scheduler: tcp://127.0.0.1:52321 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 2.00 GB


In [4]:
# FIXME examples

In [5]:
client.close()

# Modeling

